# Ground state solvers

## Introduction

<img src="aux_files/H2_gs.png" width="200">

In this tutorial we are going to discuss the ground state calculation interface of Qiskit Nature. The goal is to compute the ground state of a molecular Hamiltonian. This Hamiltonian can for example be electronic or vibrational. To know more about the preparation of the Hamiltonian, check out the [Electronic structure](01_electronic_structure.ipynb) and [Vibrational structure tutorials](02_vibrational_structure.ipynb). 

> It should be said, that in the electronic case, we are actually computing purely the **electronic** part. When using the Qiskit Nature stack as presented in this tutorial, the _nuclear repulsion energy_ will be added automatically, to obtain the **total** ground state energy.

The first step is to define the molecular system. In the following we ask for the electronic part of a hydrogen molecule.

In [1]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

/opt/hostedtoolcache/Python/3.8.18/x64/lib/python3.8/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


We will also be sticking to the Jordan-Wigner mapping. To learn more about the various mappers available in Qiskit Nature, check out the [Qubit Mappers tutorial](06_qubit_mappers.ipynb).

In [2]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()

## The Solver

After these steps, we need to define a solver. The solver is the algorithm through which the ground state is computed. 

Let's first start with a purely classical example: the `NumPyMinimumEigensolver`. This algorithm exactly diagonalizes the Hamiltonian. Although it scales badly, it can be used on small systems to check the results of the quantum algorithms. 

In [3]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()

To find the ground state we could also use the Variational Quantum Eigensolver (VQE) algorithm. The VQE algorithm works by exchanging information between a classical and a quantum computer as depicted in the following figure.

<img src="aux_files/vqe.png" width="600">

Let's initialize a VQE solver.

In [4]:
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

To define the VQE solver one needs three essential elements:

1. An Estimator primitive: these were released as part of Qiskit Terra 0.22. To learn more about primitives, check out [this resource](https://qiskit.org/documentation/apidoc/primitives.html).
2. A variational form: here we use the Unitary Coupled Cluster (UCC) ansatz (see for instance [Physical Review A 98.2 (2018): 022322]). Since it is a chemistry standard, a factory is already available allowing a fast initialization of a VQE with UCC. The default is to use all single and double excitations. However, the excitation type (S, D, SD) as well as other parameters can be selected. We also prepend the `UCCSD` variational form with a `HartreeFock` initial state, which initializes the occupation of our qubits according to the problem which we are trying solve.
3. An optimizer: this is the classical piece of code in charge of optimizing the parameters in our variational form. See [the corresponding documentation](https://qiskit.org/ecosystem/algorithms/apidocs/qiskit_algorithms.optimizers.html) for more information.

One could also use any available ansatz / initial state or even define one's own. For instance,

In [5]:
from qiskit_algorithms import VQE
from qiskit.circuit.library import TwoLocal

tl_circuit = TwoLocal(
    rotation_blocks=["h", "rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=2,
    parameter_prefix="y",
)

another_solver = VQE(Estimator(), tl_circuit, SLSQP())

## The calculation and results

We are now ready to put everything together to compute the ground-state of our problem.
Doing so requires us to wrap our `mapper` and quantum algorithm into a single `GroundStateEigensolver` like so:

In [6]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)

This will now take of the entire workflow:
1. generating the second-quantized operators stored in our problem (here referred to as `es_problem`)
2. mapping (and potentially reducing) the operators in the qubit space
3. running the quantum algorithm on the Hamiltonian qubit operator
4. once converged, evaluating the additional observables at the determined ground state

In [7]:
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030145
  - computed part:      -1.857275030145
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035696
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948930893]
    - computed part:      [0.0  0.0  1.388948930893]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000230893]  Total: 0.000000230893
                 (debye): [0.0  0.0  -0.000000586872]  Total: 0.000000586872
 


We can compare the VQE results to the NumPy exact solver and see that they match. 

In [8]:
calc = GroundStateEigensolver(mapper, numpy_solver)
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701555]
    - computed part:      [0.0  0.0  1.388948701555]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001555]  Total: 0.000000001555
                 (debye): [0.0  0.0  -0.000000003953]  Total: 0.000000003953
 


## Using a filter function

Sometimes the true ground state of the Hamiltonian is not of interest because it lies in a different symmetry sector of the Hilbert space. In this case the `NumPyEigensolver` can take a filter function to return only the eigenstates with for example the correct number of particles. This is of particular importance in the case of vibrational structure calculations where the true ground state of the Hamiltonian is the vacuum state. A default filter function to check the number of particles is implemented in the different problems and can be used as follows:

In [9]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.drivers import GaussianForcesDriver
from qiskit_nature.second_q.mappers import DirectMapper
from qiskit_nature.second_q.problems import HarmonicBasis

driver = GaussianForcesDriver(logfile="aux_files/CO2_freq_B3LYP_631g.log")
basis = HarmonicBasis([2, 2, 2, 2])
vib_problem = driver.run(basis=basis)
vib_problem.hamiltonian.truncation_order = 2

mapper = DirectMapper()

solver_without_filter = NumPyMinimumEigensolver()
solver_with_filter = NumPyMinimumEigensolver(
    filter_criterion=vib_problem.get_default_filter_criterion()
)

gsc_wo = GroundStateEigensolver(mapper, solver_without_filter)
result_wo = gsc_wo.solve(vib_problem)

gsc_w = GroundStateEigensolver(mapper, solver_with_filter)
result_w = gsc_w.solve(vib_problem)

print(result_wo)
print("\n\n")
print(result_w)

=== GROUND STATE ===
 
* Vibrational ground state energy (cm^-1): -4e-12
The number of occupied modals for each mode is: 
- Mode 0: 0.0
- Mode 1: 0.0
- Mode 2: 0.0
- Mode 3: 0.0



=== GROUND STATE ===
 
* Vibrational ground state energy (cm^-1): 2432.10695403655
The number of occupied modals for each mode is: 
- Mode 0: 1.0
- Mode 1: 1.0
- Mode 2: 1.0
- Mode 3: 1.0


In [10]:
import qiskit.tools.jupyter

%qiskit_version_table
%qiskit_copyright